# LGBM: Fake or Real - The Impostor Hunt in Texts 🔍


In [1]:
import os
import pandas as pd
import unicodedata

import string
from sklearn.metrics import accuracy_score
import numpy as np


---

### 📄 Load the Data

Now, let's load the data into memory for exploration and processing.

We'll use `Pandas` to read the file into a DataFrame, which allows for easy data manipulation and analysis throughout the notebook.

In [2]:
def read_texts_from_dir(dir_path):
  """
  Reads the texts from a given directory and saves them in the pd.DataFrame with columns ['id', 'file_1', 'file_2'].

  Params:
    dir_path (str): path to the directory with data
  """
  # Count number of directories in the provided path
  dir_count = sum(os.path.isdir(os.path.join(root, d)) for root, dirs, _ in os.walk(dir_path) for d in dirs)
  data=[0 for _ in range(dir_count)]
  print(f"Number of directories: {dir_count}")

  # For each directory, read both file_1.txt and file_2.txt and save results to the list
  i=0
  for folder_name in sorted(os.listdir(dir_path)):
    folder_path = os.path.join(dir_path, folder_name)
    if os.path.isdir(folder_path):
      try:
        with open(os.path.join(folder_path, 'file_1.txt'), 'r', encoding='utf-8') as f1:
          text1 = f1.read().strip()
        with open(os.path.join(folder_path, 'file_2.txt'), 'r', encoding='utf-8') as f2:
          text2 = f2.read().strip()
        index = int(folder_name[-4:])
        data[i]=(index, text1, text2)
        i+=1
      except Exception as e:
        print(f"Error reading directory {folder_name}: {e}")

  # Change list with results into pandas DataFrame
  df = pd.DataFrame(data, columns=['id', 'file_1', 'file_2']).set_index('id')
  return df

In [3]:
# 检查输入目录的内容
print("Input directory contents:")
print(os.listdir('/kaggle/input'))

Input directory contents:
['fake-or-real-the-impostor-hunt']


In [4]:
# Use the above function to load both train and test data
train_path="/kaggle/input/fake-or-real-the-impostor-hunt/data/train"
df_train=read_texts_from_dir(train_path)
test_path="/kaggle/input/fake-or-real-the-impostor-hunt/data/test"
df_test=read_texts_from_dir(test_path)

Number of directories: 95
Number of directories: 1068


In [5]:
df_train.head()

,file_1,file_2
id,,
0,The VIRSA (Visible Infrared Survey Telescope A...,The China relay network has released a signifi...
1,China\nThe goal of this project involves achie...,The project aims to achieve an accuracy level ...
2,Scientists can learn about how galaxies form a...,Dinosaur eggshells offer clues about what dino...
3,China\nThe study suggests that multiple star s...,The importance for understanding how stars evo...
4,Dinosaur Rex was excited about his new toy set...,Analyzing how fast stars rotate within a galax...


In [6]:
df_test.head()

,file_1,file_2
id,,
0,"""Music"" Music music music Music music Music mu...",Since its launch on Paranal observatory's Very...
1,underground exploration on SN's birth has prov...,SN 1987A provides valuable insights as newer o...
2,This research aimed to understand how star sha...,ChromeDriver music player\nThis study focused ...
3,Using OmegaCAM's wide field capabilities spann...,"greek translation :\nvazhi (megaCAM), territor..."
4,AssemblyCulture AssemblyCulture AssemblyCultur...,XClass is software tool that helps astronomers...


---

### 🏷️ Read the Labels

Next, we’ll load the **labels** associated with each text sample.
These labels indicate which text is **Real** - 1 or 2. The labels will serve as our ground truth for evaluation.

We’ll again use `Pandas` to read the label file into a DataFrame and inspect its structure.


In [7]:
# Load ground truth for train data
df_train_gt=pd.read_csv("/kaggle/input/fake-or-real-the-impostor-hunt/data/train.csv")
df_train_gt

,id,real_text_id
0,0,1
1,1,2
2,2,1
3,3,2
4,4,2
...,...,...
90,90,2
91,91,1
92,92,2
93,93,2


In [8]:
#安装和导入升级所需的库
!pip install lightgbm sentence-transformers nltk
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sentence_transformers import SentenceTransformer
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch
import re
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

2025-08-20 09:29:05.872423: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755682146.126395      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755682146.196973      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
# 下载NLTK的punkt分词器（用于句子分割）
nltk.download('punkt')

# 初始化模型（全局变量，只需加载一次）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载困惑度模型
print("加载GPT-2模型用于计算困惑度...")
perp_model_id = "gpt2"
perp_tokenizer = GPT2TokenizerFast.from_pretrained(perp_model_id)
perp_model = GPT2LMHeadModel.from_pretrained(perp_model_id).to(device)
perp_model.eval()

# 加载语义嵌入模型
print("加载SentenceTransformer模型用于计算语义一致性...")
embedder = SentenceTransformer('all-MiniLM-L6-v2')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


加载GPT-2模型用于计算困惑度...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
def extract_enhanced_features(text):
    """
    为输入的文本计算一系列特征，包括基础统计特征、困惑度和语义一致性。
    返回一个特征字典。
    """
    features = {}
    
    # 1. 基础统计特征 (原有特征)
    words = text.split()
    sentences = sent_tokenize(text)
    num_chars = len(text)
    num_words = len(words)
    num_sentences = len(sentences)
    
    features['num_chars'] = num_chars
    features['num_words'] = num_words
    features['num_sentences'] = num_sentences
    features['avg_word_length'] = np.mean([len(word) for word in words]) if num_words > 0 else 0
    features['avg_sentence_length_words'] = num_words / num_sentences if num_sentences > 0 else 0
    features['avg_sentence_length_chars'] = num_chars / num_sentences if num_sentences > 0 else 0
    
    # 词汇多样性 (Type-Token Ratio)
    unique_words = set(words)
    features['vocab_richness'] = len(unique_words) / num_words if num_words > 0 else 0
    
    # 标点符号比例
    punctuation_count = sum(1 for char in text if char in '.,!?;:')
    features['punctuation_ratio'] = punctuation_count / num_chars if num_chars > 0 else 0
    
    # 大写字母比例
    upper_case_ratio = sum(1 for char in text if char.isupper()) / num_chars if num_chars > 0 else 0
    features['upper_case_ratio'] = upper_case_ratio
    
    # 特殊字符比例 (非字母数字空格)
    special_char_ratio = sum(1 for char in text if not char.isalnum() and not char.isspace()) / num_chars if num_chars > 0 else 0
    features['special_char_ratio'] = special_char_ratio

    # 字母字符比例
    alpha_ratio = sum(1 for char in text if char.isalpha()) / num_chars if num_chars > 0 else 0
    features['alpha_ratio'] = alpha_ratio
    
    # 2. 困惑度特征 (新增)
    # 首先检查文本是否为空或过短
    if num_words < 5:  # 如果文本太短，直接设置一个高困惑度值
        features['perplexity'] = 1000  # 高困惑度表示文本异常
    else:
        max_length = 1024  # GPT-2的最大长度
        if num_chars > max_length:
            text_to_calculate = text[:max_length]
        else:
            text_to_calculate = text
            
        try:
            with torch.no_grad():
                encodings = perp_tokenizer(text_to_calculate, return_tensors="pt", truncation=True, max_length=1024)
                
                # 检查编码后的输入是否为空
                if encodings.input_ids.numel() == 0:
                    features['perplexity'] = 1000
                else:
                    input_ids = encodings.input_ids.to(device)
                    
                    # 使用模型计算损失
                    outputs = perp_model(input_ids, labels=input_ids)
                    loss = outputs.loss
                    perplexity = torch.exp(loss).item()
                    features['perplexity'] = perplexity
        except Exception as e:
            print(f"计算困惑度时出错: {e}, 文本长度: {len(text)}")
            features['perplexity'] = 1000  # 设置一个默认的高值
    
    # 3. 语义一致性特征 (新增)
    if num_sentences > 1:
        try:
            # 嵌入所有句子
            sentence_embeddings = embedder.encode(sentences)
            # 计算余弦相似度矩阵
            sim_matrix = cosine_similarity(sentence_embeddings)
            # 取非对角线元素的平均值（即不同句子间的相似度）
            np.fill_diagonal(sim_matrix, np.nan)
            mean_similarity = np.nanmean(sim_matrix)
        except Exception as e:
            print(f"计算语义一致性时出错: {e}")
            mean_similarity = 0.5  # 默认值
    else:
        mean_similarity = 1.0  # 只有一个句子，一致性为1
        
    features['semantic_consistency'] = mean_similarity
    
    return features

In [11]:
def extract_features_batch(texts, batch_size=50):
    """批量提取特征，更高效"""
    all_features = []
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_features = []
        
        for j, text in enumerate(batch_texts):
            try:
                features = extract_enhanced_features(text)
                batch_features.append(features)
            except Exception as e:
                print(f"处理文本 {i+j} 时出错: {e}")
                # 创建一个包含默认值的特征字典
                default_features = {
                    'num_chars': len(text),
                    'num_words': len(text.split()),
                    'num_sentences': len(sent_tokenize(text)),
                    'avg_word_length': 0,
                    'avg_sentence_length_words': 0,
                    'avg_sentence_length_chars': 0,
                    'vocab_richness': 0,
                    'punctuation_ratio': 0,
                    'upper_case_ratio': 0,
                    'special_char_ratio': 0,
                    'alpha_ratio': 0,
                    'perplexity': 1000,  # 高困惑度表示异常
                    'semantic_consistency': 0.5  # 中等一致性
                }
                batch_features.append(default_features)
        
        all_features.extend(batch_features)
        
        # 显示进度
        if (i // batch_size) % 10 == 0:
            print(f"已处理 {min(i+batch_size, len(texts))} / {len(texts)} 个文本")
    
    return pd.DataFrame(all_features)

In [12]:
# ---- 为训练和测试数据提取特征 ----
print("为训练数据提取特征...")

# 收集所有文本
all_train_texts = list(df_train['file_1']) + list(df_train['file_2'])
all_test_texts = list(df_test['file_1']) + list(df_test['file_2'])

# 批量提取特征
train_features_df = extract_features_batch(all_train_texts)
test_features_df = extract_features_batch(all_test_texts)

# 分割回file_1和file_2
num_train = len(df_train)
train_features_1 = train_features_df.iloc[:num_train]
train_features_2 = train_features_df.iloc[num_train:2*num_train]

num_test = len(df_test)
test_features_1 = test_features_df.iloc[:num_test]
test_features_2 = test_features_df.iloc[num_test:2*num_test]

# 获取特征列名
feature_columns = train_features_1.columns.tolist()
print(f"使用的特征: {feature_columns}")

为训练数据提取特征...


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

使用的特征: ['num_chars', 'num_words', 'num_sentences', 'avg_word_length', 'avg_sentence_length_words', 'avg_sentence_length_chars', 'vocab_richness', 'punctuation_ratio', 'upper_case_ratio', 'special_char_ratio', 'alpha_ratio', 'perplexity', 'semantic_consistency']


In [13]:
feature_columns


['num_chars',
 'num_words',
 'num_sentences',
 'avg_word_length',
 'avg_sentence_length_words',
 'avg_sentence_length_chars',
 'vocab_richness',
 'punctuation_ratio',
 'upper_case_ratio',
 'special_char_ratio',
 'alpha_ratio',
 'perplexity',
 'semantic_consistency']

In [14]:
# 保存特征
train_features_1.to_csv('train_features_1.csv', index=False)
train_features_2.to_csv('train_features_2.csv', index=False)
test_features_1.to_csv('test_features_1.csv', index=False)
test_features_2.to_csv('test_features_2.csv', index=False)

# 加载特征
# train_features_1 = pd.read_csv('train_features_1.csv')
# ...

In [15]:
# ---- 构建Learning to Rank的训练数据 ---- 
# 对于每一对，我们计算特征差异：Text_1的特征 - Text_2的特征
X_train_diff = train_features_1[feature_columns].values - train_features_2[feature_columns].values
# 我们的标签：如果真实文本是第一个，标签为1；如果是第二个，标签为0。
y_train = (df_train_gt['real_text_id'].values == 1).astype(int)

# 划分训练集和验证集
X_train, X_val, y_train_split, y_val = train_test_split(
    X_train_diff, y_train, test_size=0.2, random_state=42, stratify=y_train
)



In [16]:
# ---- 训练LightGBM模型 ----
print("训练LightGBM模型...")
# 定义模型参数
params = {
    'objective': 'binary',
    'metric': 'binary_logloss', 
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.7,  # 每次迭代使用70%的特征
    'bagging_fraction': 0.7,  # 每次迭代使用70%的数据
    'bagging_freq': 5,        # 每5次迭代进行一次bagging
    'min_data_in_leaf': 20,   # 叶子节点最少样本数
    'lambda_l1': 0.1,         # L1正则化
    'lambda_l2': 0.1,         # L2正则化
    'verbosity': -1,
    'random_state': 42,
}

# 创建LightGBM数据集
lgb_train = lgb.Dataset(X_train, label=y_train_split)
lgb_val = lgb.Dataset(X_val, label=y_val, reference=lgb_train)

# 训练模型
num_round = 200
bst = lgb.train(params,
                lgb_train,
                num_round,
                valid_sets=[lgb_val],
                callbacks=[lgb.early_stopping(stopping_rounds=20), lgb.log_evaluation(50)])



训练LightGBM模型...
Training until validation scores don't improve for 20 rounds
[50]	valid_0's binary_logloss: 0.643559
Early stopping, best iteration is:
[35]	valid_0's binary_logloss: 0.584934


In [17]:
#在训练集上评估模型 
# 预测的是Text_1优于Text_2的概率
y_pred_prob_train = bst.predict(X_train_diff, num_iteration=bst.best_iteration)
# 将概率转换为最终选择：如果概率>0.5，我们选择Text_1(预测为1)，否则选择Text_2(预测为2)
y_pred_train = np.where(y_pred_prob_train > 0.5, 1, 2)

train_accuracy = accuracy_score(df_train_gt['real_text_id'], y_pred_train)
print(f"\n增强版Learning to Rank模型在训练集上的准确率: {train_accuracy:.4f}")


增强版Learning to Rank模型在训练集上的准确率: 0.8526


In [22]:
# 使用最佳迭代对验证集进行预测
val_pred_proba = bst.predict(X_val, num_iteration=bst.best_iteration)

# 将概率转换为类别预测
val_pred = np.where(val_pred_proba > 0.5, 1, 0)

# 计算验证集准确性
val_accuracy = accuracy_score(y_val, val_pred)
print(f"验证集准确性: {val_accuracy:.4f}")

# 计算其他指标
from sklearn.metrics import classification_report, confusion_matrix

print("分类报告:")
print(classification_report(y_val, val_pred))

print("混淆矩阵:")
print(confusion_matrix(y_val, val_pred))

验证集准确性: 0.7368
分类报告:
              precision    recall  f1-score   support

           0       0.73      0.80      0.76        10
           1       0.75      0.67      0.71         9

    accuracy                           0.74        19
   macro avg       0.74      0.73      0.73        19
weighted avg       0.74      0.74      0.74        19

混淆矩阵:
[[8 2]
 [3 6]]


In [18]:


# ---- 对测试集进行预测 ---- 
X_test_diff = test_features_1[feature_columns].values - test_features_2[feature_columns].values
y_pred_prob_test = bst.predict(X_test_diff, num_iteration=bst.best_iteration)
predictions_test_ltr = np.where(y_pred_prob_test > 0.5, 1, 2)



In [19]:
# ---- 准备提交文件 ---- 
output_df = pd.DataFrame({
    'id': range(len(predictions_test_ltr)),
    'real_text_id': predictions_test_ltr
})
output_df.to_csv('ltrV2_submission.csv', index=False)
print("提交文件 'ltrV2_submission.csv' 已保存！")


提交文件 'ltrV2_submission.csv' 已保存！


In [20]:
output_df

,id,real_text_id
0,0,2
1,1,1
2,2,1
3,3,2
4,4,1
...,...,...
1063,1063,1
1064,1064,1
1065,1065,2
1066,1066,2


In [27]:
# 查看特征重要性
importance = bst.feature_importance(importance_type='split')

feature_names = feature_columns

# 创建特征重要性DataFrame
feat_imp = pd.DataFrame({
    'feature': feature_names,
    'importance': importance
}).sort_values('importance', ascending=False)

print(feat_imp)

                      feature  importance
2               num_sentences          16
0                   num_chars           8
1                   num_words           6
11                 perplexity           3
7           punctuation_ratio           1
9          special_char_ratio           1
3             avg_word_length           0
4   avg_sentence_length_words           0
5   avg_sentence_length_chars           0
6              vocab_richness           0
8            upper_case_ratio           0
10                alpha_ratio           0
12       semantic_consistency           0


# RF

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


In [30]:
# 优化的随机森林参数
rf_params = {
    'n_estimators': 200,           # 增加树的数量以提高稳定性
    'max_depth': 15,               # 限制树的最大深度，防止过拟合
    'min_samples_split': 10,       # 节点分裂所需的最小样本数
    'min_samples_leaf': 4,         # 叶节点所需的最小样本数
    'max_features': 'sqrt',        # 每次分裂时考虑的特征数 (sqrt(n_features))
    'bootstrap': True,             # 使用自助采样(bootstrap)
    'oob_score': True,             # 使用袋外样本评估模型
    'random_state': 42,            # 确保结果可重现
    'n_jobs': -1,                  # 使用所有可用的CPU核心
}

In [31]:
# 创建随机森林模型
rf_model = RandomForestClassifier(**rf_params)

# 训练模型
rf_model.fit(X_train, y_train_split)

RandomForestClassifier(max_depth=15, min_samples_leaf=4, min_samples_split=10,
                       n_estimators=200, n_jobs=-1, oob_score=True,
                       random_state=42)

In [32]:
# 评估模型性能
# 1. 使用袋外分数（OOB Score）
print(f"袋外分数 (OOB Score): {rf_model.oob_score_:.4f}")

# 2. 交叉验证评估
cv_scores = cross_val_score(rf_model, X_train, y_train_split, cv=5, scoring='accuracy')
print(f"交叉验证准确率: {cv_scores.mean():.4f} (±{cv_scores.std()*2:.4f})")

# 3. 在验证集上评估
val_pred = rf_model.predict(X_val)
val_accuracy = accuracy_score(y_val, val_pred)
print(f"验证集准确率: {val_accuracy:.4f}")

# 4. 在训练集上评估（查看过拟合程度）
train_pred = rf_model.predict(X_train)
train_accuracy = accuracy_score(y_train_split, train_pred)
print(f"训练集准确率: {train_accuracy:.4f}")
print(f"训练集-验证集差距: {train_accuracy - val_accuracy:.4f}")

袋外分数 (OOB Score): 0.8026
交叉验证准确率: 0.8150 (±0.1035)
验证集准确率: 0.7368
训练集准确率: 0.9079
训练集-验证集差距: 0.1711


In [33]:
# 5. 特征重要性
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n特征重要性:")
print(feature_importance.head(10))


特征重要性:
                      feature  importance
2               num_sentences    0.224970
1                   num_words    0.169805
0                   num_chars    0.149140
9          special_char_ratio    0.133106
11                 perplexity    0.071881
12       semantic_consistency    0.048591
10                alpha_ratio    0.039194
3             avg_word_length    0.033197
6              vocab_richness    0.029746
5   avg_sentence_length_chars    0.028418


In [34]:
# 对测试集进行预测 
y_pred_test_rf = rf_model.predict(X_test_diff)
predictions_test= np.where(y_pred_test_rf > 0.5, 1, 2)
# ---- 准备提交文件 ---- 
output_df = pd.DataFrame({
    'id': range(len(predictions_test)),
    'real_text_id': predictions_test
})
output_df.to_csv('rf_submission.csv', index=False)
print("提交文件 'rf_submission.csv' 已保存！")

提交文件 'rf_submission.csv' 已保存！
